# 인스타그램 로그인, 해시태그 검색

In [1]:
# 필요한 모듈
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import re 
import json 
import pandas as pd
import urllib.request

In [2]:
keyword = '거제핫플'

# 사전 정보 정의
username = 'javan__22'
userpw = 'javan222!'
time.sleep(3)

#해시태그 url 값
url  = "https://www.instagram.com/explore/tags/{}/".format(keyword)

# 인스타 로그인 URL
loginUrl = 'https://www.instagram.com/accounts/login/'

# Chrome driver 실행
driver = webdriver.Chrome("data4/chromedriver.exe")
driver.get(loginUrl)
time.sleep(5)

# login
driver.find_element_by_name('username').send_keys(username)
driver.find_element_by_name('password').send_keys(userpw)
time.sleep(2)
driver.find_element_by_xpath("""//*[@id="loginForm"]/div/div[3]/button""").click()
time.sleep(3)
#정보 나중에 저장하기 클릭하고 넘어가기
driver.find_element_by_xpath("""//*[@id="react-root"]/section/main/div/div/div/div/button""").click()
time.sleep(3)
#설정 나중에하기 클릭하고 넘어가기
driver.find_element_by_xpath("""/html/body/div[5]/div/div/div/div[3]/button[2]""").click()

#해시태그 검색창에 'keyword'검색
driver.get(url)
time.sleep(3)

# 인스타그램 게시물 내용 크롤링

In [4]:
# time.sleep()은 정보를 들고 오기까지의 일정 시간을 정해주기 위해 사용
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

In [5]:
# 한페이지에서 크롤링하고 다음 페이지로 넘어가는 코드를 작성
def move_next(driver):

    right = driver.find_element_by_css_selector (
        'body > div._2dDPU.QPGbb.CkGkG > div.EfHg9 > div > div > div.l8mY4 > button > div')
    right.click()
    time.sleep(3)

move_next(driver)

In [6]:
# 인스타그램 페이지에서 F12버튼을 눌러 path를 확인하고 해시태그, 작성일자를 가져와서 저장
def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    try:
        tag = re.findall(r'#[^\s#,\\]+', content)
    except:
        tag = ' '
    # 작성일자 
    try :
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    except :
        date = ' '

    data = [date, tag]
    return data

get_content(driver)

['2021-11-02',
 ['#바다뷰',
  '#물멍)',
  '#거제맛집',
  '#거제도맛집',
  '#거제맛집추천',
  '#거제맛집베스트',
  '#거제맛집추천',
  '#거제핫플',
  '#거제가볼만한곳',
  '#거제여행',
  '#거제여행코스',
  '#거제카페',
  '#거제도카페']]

In [7]:
# 한번에 최대 300까지 크롤링 가능해서 target을 300주고 크롤링 실시!
results = []

target = 3
for i in range(target):
    try:
        data = get_content(driver)  
        results.append(data)
        move_next(driver)
    except:
        time.sleep(0.5)
        move_next(driver)
    
print(results)

[['2021-11-02', ['#바다뷰', '#물멍)', '#거제카페', '#거제도카페', '#거제카페추천', '#거제도카페추천', '#거제핫플', '#거제가볼만한곳', '#거제여행', '#거제여행코스', '#거제마소마레', '#마소마레', '#거제맛집', '#거제도맛집']], ['2021-11-02', ['#바다뷰', '#물멍)', '#거제맛집', '#거제도맛집', '#거제맛집추천', '#거제맛집베스트', '#거제맛집추천', '#거제핫플', '#거제가볼만한곳', '#거제여행', '#거제여행코스', '#거제카페', '#거제도카페']], ['2021-10-30', ['#거제여행', '#거제가볼만한곳', '#거제맛집', '#거제핫플', '#거제도여행', '#거제도핫플', '#거제도맛집', '#중곡동맛집', '#고현맛집', '#통영맛집', '#장승포맛집', '#둔덕맛집', '#장목맛집', '#상동맛집']]]


In [8]:
len(results)

3

In [9]:
# 해시태그 같은경우 날짜와 해시태그를 크롤링한후 엑셀파일에 저장
sample = pd.DataFrame(results)
sample.columns = ['날짜','해시태그']
sample.to_excel('data_p/거제핫플1.xlsx', index=False, encoding='UTF-8')

# 300개씩 크롤링 한 파일을 펜션1, 펜션2, ``` 방식으로 저장
# 크롤링이 다 끝난 뒤에 파일을 하나로 합쳐줌.(엑셀 복사 붙여넣기로 작업함)
# 합친 파일 : 'data_p/거제핫플.xlsx'

# 인스타그램 게시물 URL 크롤링

In [10]:
# 게시물 클릭하지 않은 상태 (https://www.instagram.com/explore/tags/검색어/)에서 실행
# 지금 코드는 url 10개 이상 가져오고 멈추기

url_list=[]
SCROLL_PAUSE_TIME = 2
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    for i in range(3):   # url이 스크롤 한 번 내릴때마다 9개씩 업데이트 되고, 이전의 9개는 사라지는 방식이며, 
                         # 한 번에 30개 좀 넘게 뜬다. 그래서 9개씩 3번 스크롤 내려서 27개씩 업데이트 후 
                         #중복된 url은 제거하는 방식을 선택
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
         # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        links= driver.find_elements_by_xpath('//*[contains(@href, "/p/")]')
    for link in links:
        try:
            time.sleep(2)
            postUrl = link.get_attribute("href")
            url_list.append(postUrl)
        except:
            pass
    
    # url 10개 이상 가져왔으면, 멈추기
    if len(url_list) >10 : break
        
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


url_list

['https://www.instagram.com/p/CVwTK_JhV1R/',
 'https://www.instagram.com/p/CVwUEPthvE1/',
 'https://www.instagram.com/p/CVop92flNuv/',
 'https://www.instagram.com/p/CVuUieiPNe3/',
 'https://www.instagram.com/p/CVunE0Jl-H6/',
 'https://www.instagram.com/p/CVvg3rDP88Z/',
 'https://www.instagram.com/p/CVu0qwQhSS-/',
 'https://www.instagram.com/p/CVugTP8l8KM/',
 'https://www.instagram.com/p/CVul0poF91b/',
 'https://www.instagram.com/p/CVwz_ZvPxR2/',
 'https://www.instagram.com/p/CVwzzO0PhxQ/',
 'https://www.instagram.com/p/CVwzjepPDkd/',
 'https://www.instagram.com/p/CVwy15NlrxQ/',
 'https://www.instagram.com/p/CVwzKMHKV27/',
 'https://www.instagram.com/p/CVwzBbQP1Ts/',
 'https://www.instagram.com/p/CVwytsDvS6Z/',
 'https://www.instagram.com/p/CVwynG8P0lB/',
 'https://www.instagram.com/p/CVwyKtwv8SO/',
 'https://www.instagram.com/p/CVwyIeIP09z/',
 'https://www.instagram.com/p/CVwxfg4PRRq/',
 'https://www.instagram.com/p/CVwxXOOFUqf/',
 'https://www.instagram.com/p/CVwxPtkP_Nm/',
 'https://

In [11]:
len(url_list)

54

In [12]:
# 테스트용으로 게시물을 3개만 크롤링 했으므로 url도 앞에 3개만 저장
url_list = url_list[:3]
url_list

['https://www.instagram.com/p/CVwTK_JhV1R/',
 'https://www.instagram.com/p/CVwUEPthvE1/',
 'https://www.instagram.com/p/CVop92flNuv/']

In [13]:
# 중복된 URL 제거 후, df_url의 row 수가 위의 2번에서 내용 크롤링한 리스트 result의 길이와 동일한지 확인
a = {}
a['url']=url_list
df_url = pd.DataFrame(a)

df_url.drop_duplicates(inplace=True)

In [21]:
len(df_url)

3

In [14]:
# 길이가 같아졌다면 인덱스를 맞춰준 후 저장한다.
df_url = df_url.reset_index(drop=True)
df_url.to_excel('data_p/insta_url.xlsx', index=False)

In [15]:
# 크롤링 한 내용과 url을 합쳐준다.-- 순서대로 크롤링되었기 때문에 순서가 맞을 것이라고 생각
insta = pd.concat([sample,df_url],axis=1)

cols = ['날짜','검색어','insta_url']
insta.columns = cols
insta

,날짜,검색어,insta_url
0,2021-11-02,"[#바다뷰, #물멍), #거제카페, #거제도카페, #거제카페추천, #거제도카페추천,...",https://www.instagram.com/p/CVwTK_JhV1R/
1,2021-11-02,"[#바다뷰, #물멍), #거제맛집, #거제도맛집, #거제맛집추천, #거제맛집베스트,...",https://www.instagram.com/p/CVwUEPthvE1/
2,2021-10-30,"[#거제여행, #거제가볼만한곳, #거제맛집, #거제핫플, #거제도여행, #거제도핫플...",https://www.instagram.com/p/CVop92flNuv/


In [16]:
# 게시물에 따른 url 맞는지 처음, 중간, 끝 게시물 열어서 확인 후 저장
insta.to_excel('data_p/instagram_url.xlsx', index=False)